<h3>This file is an example of an Employee encrypting their salary using Homomorphic Encryption and subsequently decrypting it after the salary has been processed by the HR Department.</h3>

<h3>Importing Libraries</h3>

In [ ]:
%pip install tenseal
import tenseal as ts
import readerandwriter

<h3>Generate Private & Public Keys</h3>

In [ ]:
context = ts.context(ts.SCHEME_TYPE.CKKS,
                        poly_modulus_degree=8192,
                        coeff_mod_bit_sizes=[60, 40, 40, 60])
context.generate_galois_keys()
context.global_scale = 2**40
secret_context = context.serialize(save_secret_key=True)
readerandwriter.write_data(f"../keys/secret.txt", secret_context)


context.make_context_public()
public_context = context.serialize()
readerandwriter.write_data(f"../keys/public.txt", public_context)

<h3>Encrypting the Salary Data</h3>

In [ ]:
# Load the context
loaded_context = ts.context_from(readerandwriter.read_data(f"../keys/secret.txt"))

salary = [1000]
salary_encrypted = ts.ckks_vector(loaded_context, salary)
readerandwriter.write_data("../output/salary_encrypted.txt", salary_encrypted.serialize())

<h3>Decrypting the new Salary Data</h3>

In [ ]:
m_proto = readerandwriter.read_data("../output/salary_new_encrypted.txt")
m = ts.lazy_ckks_vector_from(m_proto)
m.link_context(loaded_context)
round(m.decrypt()[0], 2)